### **1. Introdução**

**Objetivo:**

Neste notebook, vamos desenvolver um pipeline de dados para extrair informações de vendas e produtos da API do MercadoLivre, armazená-las em um banco de dados SQLite e realizar uma extração adicional de dados descritivos dos produtos. O foco será nos seguintes campos:


- ID do vendedor (seller_id)
- Data da venda (date_created)
- ID do produto (item_id)
- Quantidade (quantity)
- Preço unitário (unit_price)

---

**Dependências:**

- requests: para fazer requisições à API do MercadoLivre.
- sqlite3: para interagir com o banco de dados SQLite.
- json: para manipulação de dados JSON.

## **2. Configuração Inicial**

- **Importação das bibliotecas necessárias:** Neste passo, vamos importar as bibliotecas essenciais para interagir com a API do MercadoLivre e manipular os dados.

In [ ]:
import requests
import sqlite3
import json
from datetime import datetime


- **Definição das variáveis de autenticação e id do vendedor:** Aqui, definimos as variáveis necessárias para autenticação na API e o vendedor que queremos consultar.


**Observação:** Para obter uma chave de acesso à API, siga os passos descritos na documentação oficial:

[Documentação de Autenticação e Autorização do MercadoLivre.](https://developers.mercadolivre.com.br/pt_br/autenticacao-e-autorizacao)

In [ ]:
ACCESS_TOKEN = 'SEU_ACCESS_TOKEN_AQUI'
SITE_ID = 'MLB' # Id referente ao Brasil
SELLER_ID = ['792353538', '451403353', '18229478']  # ID da loja oficial da Nike, Adidas e Futnatics

### **3. Criação do banco de dados e da tabela**

- **Objetivo:**
Neste passo, vamos criar uma tabela para armazenar os dados de vendas extraídos da API do Mercado Livre. A tabela será nomeada **vendas**, e sua estrutura será cuidadosamente projetada para garantir a integridade e a eficiência do banco de dados.


In [ ]:
conn = sqlite3.connect('mercadolivre.db')

### **3.1. Criação da Tabela para Dados de Vendas**

A tabela **vendas** será composta pelos seguintes campos:

- **id**: Um identificador único gerado automaticamente para cada venda.
 Facilita a identificação única de cada registro de venda, permitindo operações de atualização e exclusão sem ambiguidade.

- **seller_id**: ID do vendedor, um identificador único para o vendedor na plataforma.
 Permite rastrear quais vendedores estão associados a cada venda, fundamental para análises de performance.

- **date_created**: Data e hora em que a venda foi realizada.
 Essencial para análises temporais, como vendas diárias, semanais ou mensais.

- **item_id**: ID do produto, utilizado como chave primária para identificar de forma única cada venda.
 Facilita a vinculação das vendas com os produtos correspondentes, essencial para análise de desempenho por produto.

- **quantity**: Quantidade vendida do produto.
 Informação crítica para análise de volume de vendas e planejamento de estoque.

 - **currency**: Moeda utilizada para cadastrar o produto.

- **unit_price**: Preço unitário do produto na venda.
 Permite calcular a receita gerada por cada venda e realizar análises financeiras.

- **catalog_product_id**: ID do produto segundo a plataforma. Usada para buscar informações com a tabela produtos.

- **created_at**: Registro de quando a venda foi inserida na tabela.
 Ajuda a rastrear quando os dados foram registrados no banco de dados, útil para auditoria e controle de versão.

- **updated_at**: Registro de quando a venda foi atualizada pela última vez.
 Melhora a rastreabilidade dos dados e auxilia na manutenção do banco, permitindo identificar alterações recentes.

- **categoria_id**: Referência para a categoria do produto.
 Permite classificar as vendas por categorias de produtos, facilitando a análise de vendas por segmento.

### **3.2 Código para Criação da Tabela vendas**

In [ ]:
cursor = conn.cursor()

create_vendas_table = """
CREATE TABLE IF NOT EXISTS vendas (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    seller_id TEXT NOT NULL,
    date_created DATETIME NOT NULL,
    item_id TEXT NOT NULL,
    quantity INTEGER NOT NULL,
    currency TEXT NOT NULL,
    unit_price REAL NOT NULL,
    catalog_product_id TEXT,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    categoria_id INTEGER,
    FOREIGN KEY (categoria_id) REFERENCES categorias(categoria_id)
);
"""

cursor.execute(create_vendas_table)

conn.commit()


#### **3.3 Justificativa da Estrutura da Tabela**

A tabela **vendas** foi estruturada com atenção às melhores práticas de banco de dados. Os campos foram escolhidos para garantir a captura de dados essenciais para análises futuras, enquanto as chaves primárias e estrangeiras estabelecem relacionamentos claros entre diferentes tabelas. A inclusão de timestamps (created_at e updated_at) permite um controle rigoroso sobre as atualizações de dados e facilita auditorias.


### **4. Popular a Tabela `vendas` com Dados da API**

#### **4.1 Estrutura do Código**

Primeiro, vamos criar um código que extrai os dados relevantes do vendedor retornados pela API e insere essas informações na tabela vendas.

In [ ]:
for SELLER in SELLER_ID:
    url = f"https://api.mercadolibre.com/sites/{SITE_ID}/search?seller_id={SELLER}"
    headers = {
        'Authorization': f'Bearer {ACCESS_TOKEN}'
    }

    response = requests.get(url, headers=headers)
    data = response.json()

    # Iterar sobre os produtos e inserir na tabela 'vendas'
    for product in data['results']:
        if product.get('catalog_listing'):
            seller_id = product['seller']['id']
            item_id = product['id']
            currency = product['currency_id']
            price = product['price']
            quantity = product['available_quantity']
            catalog_product_id = product.get('catalog_product_id')
            date_created = product['sale_price']['conditions']['start_time'] if product['sale_price']['conditions'].get('start_time') else product['stop_time']
            created_at = datetime.now()
            updated_at = datetime.now()

            insert_query = """
            INSERT INTO vendas (seller_id, date_created, item_id, quantity, currency, unit_price, catalog_product_id, created_at, updated_at)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
            """
            cursor.execute(insert_query, (seller_id, date_created, item_id, quantity, currency, price, catalog_product_id, created_at, updated_at))


conn.commit()

### **4.2 Explicação do Código**

1. **Requisição à API:** O código começa fazendo uma requisição à API do Mercado Livre para obter produtos do vendedor especificado.

2. **Conexão ao Banco de Dados:** Em seguida, estabelece uma conexão com o banco de dados SQLite.

3. **Inserção dos Dados:**
   - Para cada produto na resposta da API, os campos **seller_id**, **item_id**, **unit_price**, **available_quantity**, **date_created**, **created_at**, e **updated_at** são extraídos.
   - Os dados são inseridos na tabela **vendas** usando uma consulta SQL de inserção.

4. **Salvar Alterações:** Após a inserção, as alterações são salvas.


### **5. Extração Adicional de Dados do Produto**

Neste passo, utilizaremos o item_id de cada produto na tabela **vendas** para realizar requisições à API do Mercado Livre, coletando informações adicionais sobre cada produto.

### **5.1. Criação do schema da tabela de Produtos**

In [ ]:
create_produtos_table = """
CREATE TABLE IF NOT EXISTS produtos (
    product_id TEXT PRIMARY KEY,
    seller_id INTEGER NOT NULL,
    name TEXT NOT NULL,
    category_id TEXT NOT NULL,
    thumbnail TEXT,
    category_domain TEXT NOT NULL,
    available_quantity INTEGER NOT NULL,
    date_created DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
"""

cursor.execute(create_produtos_table)

### 5.2. Descrição Técnica dos Campos da Tabela Produtos

- **product_id**: Identificador único do produto obtido da API do Mercado Livre, utilizado como chave primária.
- **seller_id**: Identificador do vendedor que oferece o produto, possibilitando análises de vendas por vendedor.
- **name**: Nome do produto para identificação e busca.
- **description**: Descrição detalhada do produto, útil para informações adicionais sobre o item.
- **category_id**: Identificador da categoria do produto, facilitando a organização e análise por categoria.
- **thumbnail**: URL da imagem do produto para representação visual e uso em interfaces de usuário.
- **category_domain**: Descrição da categoria de acordo com o código.
- **available_quantity**: Quantidade disponível do produto.
- **date_created**: Data de criação do registro do produto na tabela, útil para auditoria e rastreamento de dados.
- **updated_at**: Data da última atualização do registro do produto.

In [ ]:
cursor.execute("SELECT item_id FROM vendas")
item_ids = cursor.fetchall()

for item in item_ids:
    item_id = item[0]  # extrai o item_id da tupla

    url = f"https://api.mercadolibre.com/items/{item_id}"
    headers = {'Authorization': f'Bearer {ACCESS_TOKEN}'}
    response = requests.get(url, headers=headers)
    product_data = response.json()

    product_id = product_data['id']
    seller_id = product_data['seller_id']
    name = product_data['title']
    category_id = product_data['category_id']
    thumbnail = product_data['thumbnail']
    category_domain = product_data['domain_id']
    available_quantity = product_data['initial_quantity']
    date_created = datetime.now()
    updated_at = datetime.now()

    cursor.execute("SELECT COUNT(1) FROM produtos WHERE product_id = ?", (product_id,))
    exists = cursor.fetchone()[0]

    if exists == 0:
        insert_query = """
        INSERT INTO produtos (product_id, seller_id, name, category_id, thumbnail, category_domain, available_quantity, date_created, updated_at)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
        """
        cursor.execute(insert_query, (product_id, seller_id, name, category_id, thumbnail, category_domain, available_quantity, date_created, updated_at))

conn.commit()

### **5.2. Explicação do Código**

1. **Conexão ao Banco de Dados:** O código se conecta ao banco de dados SQLite, onde a tabela **produtos** foi criada.

2. **Recuperação de item_ids:** Recupera todos os items da tabela **vendas** para que os dados descritivos possam ser buscados.

3. **Requisição à API:** Para cada item_id, o código faz uma nova requisição à API do Mercado Livre para obter detalhes adicionais do produto.

4. **Inserção dos Dados:** Os dados coletados são inseridos na tabela **produtos** através de uma consulta SQL.

5. **Salvar Alterações:** As alterações são salvas no banco de dados.


### **6. Como você nomearia cada tabela e por quê?**

### Tabela vendas

**Justificativa**:
1. **Clareza**: O nome vendas é direto e descritivo, permitindo que qualquer membro da equipe de dados compreenda imediatamente o propósito da tabela. Essa clareza é vital em um ambiente de trabalho onde a equipe pode incluir profissionais de diferentes formações e níveis de experiência.

2. **Consistência**: A nomenclatura segue uma convenção de nomeação que é facilmente reconhecível. Isso é particularmente importante em um banco de dados com várias tabelas, onde manter uma consistência nos nomes facilita a navegação e o entendimento geral da estrutura de dados.

3. **Escalabilidade**: À medida que a empresa cresce e mais dados são coletados, a tabela vendas pode ser expandida para incluir novas colunas ou relacionamentos sem a necessidade de alterar seu nome ou estrutura básica. Isso é essencial em um ambiente corporativo onde os requisitos de negócios podem mudar rapidamente.

### Tabela produtos

**Justificativa**:
1. **Clareza**: Assim como a tabela vendas, o nome produtos comunica de forma eficaz o conteúdo da tabela. Ele reflete precisamente que as informações contidas são relacionadas a produtos, essencial para consultas e relatórios que a equipe de negócios possa necessitar.

2. **Facilidade de consulta**: Em um ambiente corporativo onde relatórios e análises de dados são comuns, ter uma tabela claramente nomeada como produtos permite que os analistas de BI construam consultas SQL rapidamente, sem a necessidade de pesquisar ou consultar documentação adicional.

3. **Relacionalidade**: A escolha do nome produtos também permite uma fácil definição de relacionamentos com outras tabelas, como a tabela vendas. Essa relação é crítica para análises que buscam entender o desempenho de vendas por produto, facilitando decisões estratégicas baseadas em dados.

### Conclusão

A escolha dos nomes das tabelas vendas e produtos reflete uma abordagem deliberada para garantir clareza, consistência e escalabilidade em um ambiente corporativo complexo. Essas características são fundamentais para facilitar a manutenção do banco de dados e a realização de análises de dados, promovendo uma cultura orientada a dados que é essencial para o sucesso da empresa. Através dessa nomenclatura cuidadosa, buscamos não apenas atender às necessidades atuais, mas também garantir que o sistema possa evoluir de forma sustentável em resposta às futuras demandas de negócios.

In [ ]:
# Fechando a conexão com o banco
conn.close()

________________________________________________
________________________________
_________________________
________
____


________________________________________________
________________________________
_________________________
________
____


## **Atividade 2**

1. A atividade 2 faz parte da análise dos dados capturados na atividade 1, foram criadas visões de receita total, número de produtos vendidos e o preço médio de venda por vendedor no banco de dados.
2. Foi identificado qual o produto mais vendido, o vendedor com maior faturamento e a evolução agregada das vendas ao longo do tempo separadas por categoria.

In [ ]:
query_receita_total = """
SELECT printf('R$ %,.2f', SUM(v.unit_price * v.quantity)) AS receita_total
FROM vendas v
WHERE v.currency = 'BRL';
"""
cursor.execute(query_receita_total)
receita_total = cursor.fetchone()[0]
print(f"Receita Total: {receita_total}")

In [ ]:
query_preco_medio_por_vendedor = """
SELECT v.seller_id, printf('R$ %,.2f', AVG(v.unit_price)) AS preco_medio
FROM vendas v
WHERE currency = 'BRL'
GROUP BY v.seller_id;
"""
cursor.execute(query_preco_medio_por_vendedor)
preco_medio_por_vendedor = cursor.fetchall()
print("Preço médio por vendedor:\n\n")
for row in preco_medio_por_vendedor:
    print(f"Vendedor ID: {row[0]}, Preço Médio: {row[1]}")

In [ ]:
query_total_produtos_vendidos = """
SELECT SUM(v.quantity) AS total_produtos_vendidos
FROM vendas v;
"""
cursor.execute(query_total_produtos_vendidos)
total_produtos_vendidos = cursor.fetchone()[0]
print(f"Total de Produtos Vendidos: {total_produtos_vendidos}")


In [ ]:
query_produto_mais_vendido = """
SELECT p.name, SUM(v.quantity) AS total_vendido
FROM vendas v
LEFT JOIN produtos p ON v.item_id = p.product_id
GROUP BY p.name
ORDER BY total_vendido DESC
LIMIT 1;
"""
cursor.execute(query_produto_mais_vendido)
produto_mais_vendido = cursor.fetchone()
print(f"Produto Mais Vendido: {produto_mais_vendido[0]}, Total Vendido: {produto_mais_vendido[1]}")

In [ ]:
query_vendedor_maior_faturamento = """
SELECT v.seller_id, printf('R$ %,.2f', SUM(v.unit_price * v.quantity)) AS faturamento_total
FROM vendas v
WHERE currency = 'BRL'
GROUP BY v.seller_id
ORDER BY faturamento_total DESC
LIMIT 1;
"""
cursor.execute(query_vendedor_maior_faturamento)
vendedor_maior_faturamento = cursor.fetchone()
print(f"Vendedor com maior faturamento:\n\n Vendedor ID: {vendedor_maior_faturamento[0]}, Faturamento Total: {vendedor_maior_faturamento[1]}")

In [ ]:
query_evolucao_vendas = """
SELECT strftime('%m-%Y', v.date_created) AS mes_ano,
       COALESCE(p.category_id, 'Desconhecida') AS categoria,
       SUM(v.quantity) AS total_vendas
FROM vendas v
LEFT JOIN produtos p ON v.item_id = p.product_id
GROUP BY mes_ano, categoria
ORDER BY total_vendas DESC;
"""
cursor.execute(query_evolucao_vendas)
evolucao_vendas = cursor.fetchall()

print("Evolução de vendas ao longo do tempo por categoria:\n")
for row in evolucao_vendas:
    print(f"Mês-Ano: {row[0]}, Categoria: {row[1]}, Total Vendido: {row[2]}")

______
____________
______
______

Atividade 3: Extração dos dados para CSV

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('mercadolivre.db')

tabelas = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

print(tabelas)

for tabela in tabelas['name']:
    df = pd.read_sql_query(f"SELECT * FROM {tabela};", conn)
    df.to_csv(f"{tabela}.csv", index=False)

conn.close()


In [ ]:
from google.colab import files


for tabela in tabelas['name']:
    files.download(f"{tabela}.csv")


### 1. Vendas por Categoria
- **Explicação**: Este gráfico apresenta a distribuição das vendas por diferentes categorias de produtos. Cada barra representa o volume total de vendas de uma categoria específica, permitindo uma comparação clara entre elas.

### 2. Análise Temporal de Vendas
- **Explicação**: Este gráfico ilustra a evolução das vendas ao longo do tempo.

### 3. Receita Total por Vendedor
- **Explicação**: Este gráfico mostra a receita total gerada por cada vendedor, permitindo uma análise da performance individual.

### 4. Quantidade de Vendas por Vendedor
- **Explicação**: Este gráfico ilustra a quantidade total de vendas realizadas por cada vendedor. Cada barra representa o total de unidades vendidas por cada vendedor.